### Step 1: Check GPU & Mount Google Drive

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

from google.colab import drive
drive.mount('/content/drive')

print("Drive mounted. Your dataset will be accessible at /content/drive/MyDrive/ChessPieceDetection")

Using device: cuda
Mounted at /content/drive
Drive mounted. Your dataset will be accessible at /content/drive/MyDrive/ChessPieceDetection


### Step 2: Clone YOLOv5 & Install Dependencies

In [ ]:
# Uninstall ultralytics if already installed (as in notebook)
!pip uninstall ultralytics -y

# Clone YOLOv5 and install requirements
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt -q
!pip install roboflow -q

print("YOLOv5 cloned and requirements installed successfully!")

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 17410, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 17410 (delta 63), reused 22 (delta 22), pack-reused 17326 (from 3)
Receiving objects: 100% (17410/17410), 16.31 MiB | 18.87 MiB/s, done.
Resolving deltas: 100% (11931/11931), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 

### Step 3: List Dataset

In [ ]:
from roboflow import Roboflow

# Download dataset from Roboflow
rf = Roboflow(api_key="THWAfydGbO78ypAyYu7H")
project = rf.workspace("cifar10-image-classification").project("chess-piece-detection-omx7i")
version = project.version(1)
dataset = version.download("yolov5")

# Get dataset path
dataset_path = dataset.location
print(f"Dataset downloaded to: {dataset_path}")

# List dataset contents
!ls "{dataset_path}"

print("\n===== Train Images =====")
!ls "{dataset_path}/train/images" | head -n 5

print("\n===== Valid Images =====")
!ls "{dataset_path}/valid/images" | head -n 5

print("\n===== Test Images =====")
!ls "{dataset_path}/test/images" | head -n 5

# Comment for clarity, as in notebook
print("\n# 'head -n 5' just shows first 5 lines, so as not to flood output.")
print("# If you only have train, that’s okay. We can still proceed.")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Chess-Piece-Detection-1 in yolov5pytorch:: 100%|██████████| 726/726 [00:00<00:00, 9973.19it/s]


Dataset downloaded to: /content/yolov5/Chess-Piece-Detection-1
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid

===== Train Images =====
00000000_0_JPG.rf.e244334b008c37821cfd063805d45c65.jpg
00000000_138_jpg.rf.7c8785d5215677936075453ecce37baa.jpg
00000000_237_jpg.rf.d4b390dcd1b290a69484f69d707b8970.jpg
00000000_75_jpg.rf.1c024186cedc86b84b1963cdc089ad63.jpg
00000001_139_jpg.rf.8270ba175688d9c8006aa63ae51b576e.jpg

===== Valid Images =====
00000174_63_jpg.rf.783a3ad6eb09da03a2b3f16b1243d3bb.jpg
00000175_323_jpg.rf.ed112cf4c8923e129f2b5474f4aeba8b.jpg
00000178_65_jpg.rf.cc3bc5099a0d066261107c3ebe82a672.jpg
00000180_326_jpg.rf.7b2d61164e60b1df27aeb79f3ec09b3b.jpg
00000182_387_jpg.rf.1d7d68c983eb50355c5ddcadfb02fdc4.jpg

===== Test Images =====
00000142_214_jpg.rf.3ddd95438c9ad351b46a89dd2b1e35fd.jpg
00000143_381_jpg.rf.b2f886b46f45b3b8454b71afa44f86b1.jpg
00000144_128_jpg.rf.bd273913b4d7fdcad5374b9a412c9860.jpg
00000145_215_jpg.rf.cc6cde4d6de27f5a357fd6d076be811e.

### Step 4: Overwrite data.yaml

In [ ]:
# Inspect labels to get unique classes
import glob
import os

def get_unique_classes(label_dir):
    class_ids = set()
    label_files = glob.glob(os.path.join(label_dir, "*.txt"))
    for label_file in label_files:
        with open(label_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                class_id = int(line.split()[0])
                class_ids.add(class_id)
    return sorted(list(class_ids))

train_label_dir = os.path.join(dataset_path, "train/labels")
unique_classes = get_unique_classes(train_label_dir)
print(f"Unique class IDs found in labels: {unique_classes}")

# Define class names (based on chess pieces)
class_names = [
    'black_bishop', 'black_king', 'black_knight', 'black_pawn', 'black_queen', 'black_rook',
    'white_bishop', 'white_king', 'white_knight', 'white_pawn', 'white_queen', 'white_rook'
]

# Verify number of classes
num_classes = len(unique_classes)
if num_classes > len(class_names):
    print(f"Warning: Found {num_classes} classes, but only {len(class_names)} names defined. Using generic names.")
    class_names = [f"class_{i}" for i in range(num_classes)]
elif num_classes < len(class_names):
    print(f"Warning: Found {num_classes} classes, using first {num_classes} names.")
    class_names = class_names[:num_classes]

# Create data.yaml
yaml_content = f"""train: {dataset_path}/train/images
val: {dataset_path}/valid/images
nc: {num_classes}
names: {class_names}"""

# Write to dataset directory, as in notebook
yaml_path = f"{dataset_path}/data.yaml"
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"Overwriting {yaml_path}")
print(yaml_content)

Unique class IDs found in labels: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Overwriting /content/yolov5/Chess-Piece-Detection-1/data.yaml
train: /content/yolov5/Chess-Piece-Detection-1/train/images
val: /content/yolov5/Chess-Piece-Detection-1/valid/images
nc: 12
names: ['black_bishop', 'black_king', 'black_knight', 'black_pawn', 'black_queen', 'black_rook', 'white_bishop', 'white_king', 'white_knight', 'white_pawn', 'white_queen', 'white_rook']


### Step 5: Train the Model

In [ ]:
%cd /content/yolov5

data_yaml_path = f"{dataset_path}/data.yaml"
cfg_path = "yolov5s.yaml"

!python train.py \
  --data {data_yaml_path} \
  --cfg {cfg_path} \
  --weights yolov5s.pt \
  --epochs 50 \
  --batch 16 \
  --img 640 \
  --name chess_piece_exp \
  --cache

/content/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-04-24 08:57:39.690864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745485059.981444    2975 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745485060.048188    2975 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for 

### Step 6: Save and Copy Model Weights

In [ ]:
# List weights
!ls runs/train/chess_piece_exp/weights/

# Zip best.pt
!zip -r best.zip runs/train/chess_piece_exp/weights/best.pt

# Copy best.pt to Google Drive
!mkdir -p /content/drive/MyDrive/ChessPieceDetection
!cp runs/train/chess_piece_exp/weights/best.pt /content/drive/MyDrive/ChessPieceDetection/

print("Best model weights saved to /content/drive/MyDrive/ChessPieceDetection/best.pt")

best.pt  last.pt
  adding: runs/train/chess_piece_exp/weights/best.pt (deflated 9%)
Best model weights saved to /content/drive/MyDrive/ChessPieceDetection/best.pt


### Step 7: Test on Video

In [ ]:
# Ensure YOLOv5 repository is set up
%cd /content
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt -q
!pip install opencv-python -q

# Import required libraries
import cv2
from pathlib import Path
from models.common import DetectMultiBackend
from utils.general import non_max_suppression
from utils.plots import Annotator, colors
import torch
import numpy as np

# Copy best.pt from Google Drive to YOLOv5 directory
!cp /content/drive/MyDrive/ChessPieceDetection/best.pt /content/yolov5/

# Load YOLOv5 model
weights_path = '/content/yolov5/best.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DetectMultiBackend(weights_path, device=device)

# Video file path (corrected from your input)
video_path = '/content/drive/MyDrive/Aivancity/Deep Learning/Chess Piece Detection.mp4'

# Open video
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Unable to open video. Please ensure '{video_path}' exists and is accessible.")
    exit()

# Video writer
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_chess.mp4', fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Prepare frame for YOLO
    img = frame[:, :, ::-1].copy()  # Convert BGR to RGB
    img = torch.from_numpy(img).to(device).permute(2, 0, 1).float() / 255.0
    img = img.unsqueeze(0)  # Add batch dimension

    # Run inference
    results = model(img)
    detections = non_max_suppression(results)

    # Annotate frame
    annotator = Annotator(frame, line_width=2, example="YOLOv5")
    for det in detections:
        if det is not None and len(det):
            for *box, conf, cls in det:
                label = f"{model.names[int(cls)]} {conf:.2f}"
                annotator.box_label(box, label, color=colors(int(cls), True))

    annotated_frame = annotator.result()
    out.write(annotated_frame)

cap.release()
out.release()
print("Output video saved as 'output_chess.mp4'")

# Download the output video
from google.colab import files
files.download('output_chess.mp4')

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


Fusing layers... 
YOLOv5s summary: 157 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs


Output video saved as 'output_chess.mp4'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>